<a href="https://colab.research.google.com/github/petrovortex/dls-homework-sem-2/blob/main/hw_language_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Э
В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

In [ ]:
!pip install datasets

Импорт необходимых библиотек

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [ ]:
dataset = load_dataset('imdb')

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предлоения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [ ]:
sentences = []
word_threshold = 32

for features in dataset['train']:
    sentences.extend(
        [sentence for sentence in sent_tokenize(features['text']) if len(word_tokenize(sentence)) < word_threshold]
    )

In [ ]:
print("Всего предложений:", len(sentences))

Посчитаем для каждого слова его встречаемость.

In [ ]:
words = Counter()

for sentence in sentences:
    words.update(word_tokenize(sentence))

Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [ ]:
vocab = set()
vocab_size = 40000

vocab.update(sorted(list(words.keys()), key=lambda word: words[word], reverse=True)[:vocab_size])

vocab.update(['<unk>', '<bos>', '<eos>', '<pad>'])


In [ ]:
assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4

In [ ]:
print("Всего слов в словаре:", len(vocab))

### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [ ]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [ ]:
class WordDataset(Dataset):
    def __init__(self, sentences):
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

        self.tokenized_data = []
        for sentence in sentences:
            words_in_sent = word_tokenize(sentence)
            ids = [word2ind.get(w, self.unk_id) for w in words_in_sent]
            tokenized_sentence = [self.bos_id] + ids + [self.eos_id]
            self.tokenized_data.append(tokenized_sentence)

    def __getitem__(self, idx: int) -> List[int]:
        return self.tokenized_data[idx]

    def __len__(self) -> int:
        return len(self.tokenized_data)


In [ ]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [ ]:
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 128

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве.

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации.
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве.

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Функция evaluate (1 балл)

Заполните функцию ```evaluate```

In [ ]:
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            logits = model(batch['input_ids']).flatten(0, 1)
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

### Train loop (1 балл)

Напишите функцию для обучения модели.

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

def train_loop(model, optimizer, criterion, train_dataloader, eval_dataloader,
               device, n_epochs, patience=3):

    history = {'train_loss': [], 'val_perplexity': []}
    best_perplexity = float('inf')
    epochs_no_improve = 0

    model.to(device)

    for epoch in range(n_epochs):
        model.train()
        epoch_loss = []

        for batch in tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{n_epochs} [Training]'):
            optimizer.zero_grad()

            logits = model(batch['input_ids']).flatten(0, 1)
            loss = criterion(logits, batch['target_ids'].flatten())

            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.item())

        avg_train_loss = sum(epoch_loss) / len(epoch_loss)
        history['train_loss'].append(avg_train_loss)

        val_perplexity = evaluate(model, criterion, eval_dataloader)
        history['val_perplexity'].append(val_perplexity)

        clear_output(wait=True)
        plt.figure(figsize=(14, 6))

        plt.subplot(1, 2, 1)
        plt.plot(history['train_loss'], label='Train Loss')
        plt.title('Training Loss')
        plt.xlabel('Epoch')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(history['val_perplexity'], label='Validation Perplexity')
        plt.title('Validation Perplexity')
        plt.xlabel('Epoch')
        plt.legend()
        plt.grid(True)

        plt.show()

        print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {avg_train_loss:.4f}, Val Perplexity: {val_perplexity:.4f}")

        if val_perplexity < best_perplexity:
            best_perplexity = val_perplexity
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {patience} epochs with no improvement.")
            break

    return history

### Первый эксперимент (2 балла)

Определите архитектуру модели и обучите её.

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers,
                 dropout_p=0.2, rnn_type='LSTM'):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim,
                                      padding_idx=word2ind['<pad>'])

        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers,
                               batch_first=True, dropout=dropout_p if num_layers > 1 else 0)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers,
                              batch_first=True, dropout=dropout_p if num_layers > 1 else 0)
        else:
            self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers,
                              batch_first=True, dropout=dropout_p if num_layers > 1 else 0)

        self.dropout = nn.Dropout(dropout_p)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        embedded = self.dropout(embedded)

        rnn_out, _ = self.rnn(embedded)

        logits = self.fc(rnn_out)

        return logits

In [ ]:
!pip install -q optuna

In [ ]:
import optuna

def objective(trial):
    rnn_type = trial.suggest_categorical('rnn_type', ['LSTM', 'GRU'])
    embedding_dim = trial.suggest_categorical('embedding_dim', [128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [256, 512])
    num_layers = trial.suggest_int('num_layers', 1, 2)
    dropout_p = trial.suggest_float('dropout_p', 0.0, 0.3)
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-2, log=True)

    model = LanguageModel(
        vocab_size=len(vocab),
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout_p=dropout_p,
        rnn_type=rnn_type
    ).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])

    N_EPOCHS_TRIAL = 5

    for epoch in range(N_EPOCHS_TRIAL):
        model.train()
        for batch in train_dataloader:
            optimizer.zero_grad()
            logits = model(batch['input_ids']).flatten(0, 1)
            loss = criterion(logits, batch['target_ids'].flatten())
            loss.backward()
            optimizer.step()

        val_perplexity = evaluate(model, criterion, eval_dataloader)

        trial.report(val_perplexity, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_perplexity

In [ ]:
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())

study.optimize(objective, n_trials=20, show_progress_bar=True)

print("Best trial:")
trial = study.best_trial

print(f"  Value (Perplexity): {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
best_params = study.best_params

final_model = LanguageModel(
    vocab_size=len(vocab),
    embedding_dim=best_params['embedding_dim'],
    hidden_dim=best_params['hidden_dim'],
    num_layers=best_params['num_layers'],
    dropout_p=best_params['dropout_p'],
    rnn_type=best_params['rnn_type']
).to(device)

final_optimizer = torch.optim.AdamW(
    final_model.parameters(), lr=best_params['learning_rate'])

criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])

train_loop(final_model, final_optimizer, criterion, train_dataloader,
           eval_dataloader, device, n_epochs=20, patience=5)


test_perplexity = evaluate(final_model, criterion, test_dataloader)
print(f'Итоговая перплексия на тестовом наборе: {test_perplexity:.4f}')

In [ ]:
import torch.nn.functional as F
from nltk.tokenize import word_tokenize

def generate_sequence(model, starting_seq: str, max_len: int = 50, temperature: float = 0.8):
    device = next(model.parameters()).device
    model.eval()

    tokenized_start = word_tokenize(starting_seq.lower())
    input_ids = [word2ind['<bos>']] + [
        word2ind.get(word, word2ind['<unk>']) for word in tokenized_start
    ]

    input_tensor = torch.LongTensor([input_ids]).to(device)

    generated_indices = list(input_ids)

    with torch.no_grad():
        for _ in range(max_len):
            logits = model(input_tensor)

            last_word_logits = logits[:, -1, :]

            if temperature > 0:
                probabilities = F.softmax(last_word_logits / temperature, dim=-1)
                next_token_id = torch.multinomial(probabilities, num_samples=1).item()
            else:
                next_token_id = torch.argmax(last_word_logits, dim=-1).item()

            if next_token_id == word2ind['<eos>']:
                break

            generated_indices.append(next_token_id)

            input_tensor = torch.LongTensor([generated_indices]).to(device)

    generated_words = [ind2word.get(idx, '<unk>') for idx in generated_indices]

    final_sequence = ' '.join(word for word in generated_words if word not in ['<bos>', '<eos>'])

    return final_sequence

In [ ]:
final_model.eval()

prompt = "this movie was about"
print("--- Greedy Search (temperature=0.0) ---")
print(generate_sequence(final_model, starting_seq=prompt, max_len=30, temperature=0.0))
print("\n" + "="*50 + "\n")

print("--- Sampling (temperature=0.5) ---")
print(generate_sequence(final_model, starting_seq=prompt, max_len=30, temperature=0.5))

### Второй эксперимент (2 балла)

Попробуйте что-то поменять в модели или в пайплайне обучения, идеи для экспериментов можно подсмотреть выше.

In [ ]:
# эксперименты провелись внутри optuna (см. выше)

### Отчет (2 балла)

Опишите проведенные эксперименты. Сравните перплексии полученных моделей. Предложите идеи по улучшению качества моделей.

Я организовал автоматические эксперименты с помощью библиотеки Optuna. От эксперимента к эксперименту варьировались:
1. Тип модели: GRU / LSTM
2. Количество слоев: 1 / 2
3. Размер эмбеддингов: 128 / 256
4. Размер hidden_dim: 256 / 512
5. Вероятность (процент) дропаута нейронов: 0.0-0.3
6. Величина шага обучения: 0.001-0.01

Анализ показал, что наибольшее влияние на метрику (перплексию) в указанных диапазонах оказывают количество слоев (1 лучше 2), а также дропаут (чем меньше, тем лучше). Думаю, стоит провести дополнительное исследование сравнивающее две модели GRU и LSTM, займусь этим когда дойдут руки :)